## 1. |mp0rt

In [1]:
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [2]:
df_list = pd.read_csv('listings.csv').set_index('id')
pd.set_option('display.max_columns', 500)

In [ ]:
df_list.columns

## 2. Wr4ng|

    ## Questionable columns: 'host_since'; max_min, min_max repeated(?), 
        #listings_count repeats(?)
    ## NLP columns: 'name', 'description', 'neighborhood_overview', 
    # 'host_about'(?),         
    

In [3]:
def wrangle(X):
    """A function to clean our data"""
    
    #Make a copy of the df
    X = X.copy()

    ## DROP unwanted columns; HiCard, urls, empty, redundant, etc.           80>|
    drop_cols=['listing_url', 'scrape_id', 'last_scraped', 'name', 'description', 
               'neighborhood_overview', 'picture_url', 'host_url',
               'host_name', 'host_since', 'host_location', 'host_about', 
               'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 
               'neighbourhood', 'neighbourhood_group_cleansed', 'bathrooms', 
               'amenities', 'calendar_updated', 'has_availability',
               'calendar_last_scraped', 'minimum_minimum_nights', 
               'maximum_minimum_nights', 'minimum_maximum_nights', 
               'maximum_maximum_nights']
    X.drop(columns=drop_cols, inplace=True)
    
    ## A FUNC and execution thereof tallying number of contacts
    def num_contact(items):
        items = items.replace('[','').replace(']','').replace("'",'').split(',')
        return len(items)
    # 
    X['host_verifications'] = X['host_verifications'].apply(num_contact)
    
    ## Changing columns to BOOL vals     
    # Convert "t/f" columns to BOOl columns.
    def strToBool(text):
      if text == 't':
        return True
      else:
        return False
    X['host_has_profile_pic'] = X['host_has_profile_pic'].apply(strToBool)
    X['host_identity_verified'] = X['host_identity_verified'].apply(strToBool)
    X['host_is_superhost'] = X['host_is_superhost'].apply(strToBool)
    X['instant_bookable'] = X['instant_bookable'].apply(strToBool)
    
    # Convert license column to BOOL column.
    def licenseToBool(text):      
        if type(text) != float: 
            return True
        else:
            return False
    X['license'] = X['license'].apply(lambda x: licenseToBool(x))  
    
    
    ## Shorten/rename as neighbourhood_cleansed' column
    X = X.rename({'neighbourhood_cleansed' : 'neighborhood'}, axis=1)
    
    ### BATH
    ## Change Bathroom column name
    X = X.rename({'bathrooms_text' : 'num_bathroom'}, axis=1)
    # Get rid of the 2 null values in the bathrooms_text column
    X = X[X['num_bathroom'].isnull() == False]
    # Create a column to specify if baths are shared or not
    def parse_baths_shared(string):
        if 'shared' in string.lower(): # .lower because there is one value to remove
            return True                # "Shared Half Bath", we do
        else:
            return False
        
    X['shared_bath'] = X['num_bathroom'].apply(parse_baths_shared)
    # Parse floats from the bathrooms_text column
    def parse_baths(text):
        if 'half-bath' in text.lower():
            return 0.5

        text = text.split(' ')
        return float(text[0])

    X['num_baths'] = X['num_bathroom'].apply(parse_baths)
    X = X.drop(columns='num_bathroom')
    ### END BATH
    
    # TODO: Price to INT
    X['price'] = X['price'].apply(lambda x: float(x.replace('$',"").replace(',','')))
        
    return X

In [ ]:
df_profile = wrangle(df_list)

In [ ]:
df_profile

In [ ]:
df_profile.head(20)

In [ ]:
df_profile.tail(20)

In [ ]:
df_profile.shape

    # TODO(?): Amenities to categorical, may want to transform to standardized

## €dα


In [ ]:
profile = ProfileReport(df_profile)
profile.to_notebook_iframe()

# m0d3|
    - Should we encode(onehot/ordinal) some columns: 'host_response_time', 'neighborhood'(prob no), 
      'property_type', 'room_type'
    - DONE: target/train/test
    - DONE: Get a baseline accuracy
    - DONE: Update pipfile with imports, commented out, above
    - TODO: build pipeline
    - TODO: run models

In [4]:
df_model = wrangle(df_list)

# Train//tesT :: :: :: SΦLi_|_

In [5]:
# setting the target for 2! train/test splits
target = 'price'

y_init = df_model[target]
X_init = df_model.drop(columns=target)

In [6]:
print(X_init.shape)
print(y_init.shape)

(3151, 47)
(3151,)


In [7]:
X_subseq, X_test, y_subseq, y_test = train_test_split(X_init, y_init, test_size=0.1, random_state=13)

In [8]:
print(X_test.shape)

(316, 47)


In [9]:
print(X_subseq.shape)
print(y_subseq.shape)

(2835, 47)
(2835,)


In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_subseq, y_subseq, test_size=0.1114, random_state=17)

In [11]:
X_val.shape

(316, 47)

In [12]:
print(X_train.shape)
print(X_val.shape)

(2519, 47)
(316, 47)


## B4SSLI3N

In [ ]:
baseline_pred = [y_train.mean()] * len(y_train)

baseline_mae = mean_absolute_error(y_train,baseline_pred)

print(baseline_mae)

## PIGH:P:LINE

In [ ]:
pipe_01 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(random_state = 13,
                           n_jobs = -2)
)

# params are a DICT
params_01 = {
    # 'key__hyperparam'(which step in pipeline you want looked at) : 'value'     
    'randomforestregressor__max_depth' : range(200, 1201, 100),
    'randomforestregressor__n_estimators' :  range(50, 351, 10),
    'randomforestregressor__max_samples' : np.arange(0.1, 0.9, 0.1)
    }

In [ ]:
model_01 = GridSearchCV(
    pipe_01,
    param_grid = params_01,
    n_jobs = -2,
    cv = 5,
    verbose = 1
)

## Stripped Model

In [13]:
X_train2 = X_train[['host_id', 'last_review', 'minimum_nights', 'latitude',
                   'minimum_nights_avg_ntm', 'first_review', 'bedrooms',
                   'number_of_reviews', 'neighborhood', 'num_baths']]

X_val2 = X_val[['host_id', 'last_review', 'minimum_nights', 'latitude',
                'minimum_nights_avg_ntm', 'first_review', 'bedrooms',
                'number_of_reviews', 'neighborhood', 'num_baths']]

In [16]:
pipe_02 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestRegressor(
        n_estimators = 250,
        max_depth = 150,
        max_samples = 0.5,
        random_state = 13,
        n_jobs = -2)
)

In [17]:
import joblib

filename = 'working_model.sav'
joblib.dump(pipe_02, filename)

['working_model.sav']

In [ ]:
model_01.fit(X_train, y_train)

In [ ]:
dir(model_01)

In [ ]:
model_01.error_score

In [ ]:
model_01.best_params_

'predict_proba'


### TODO: check cardinality/duplicates in neighborhood, property type, room_type (?)